In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
print(tf.__version__)
%matplotlib inline

/Users/wassim/anaconda/anaconda3/envs/deeplearning/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


1.5.0


In [2]:
CIFAR_DIR = 'cifar-10-batches-py/'

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        cifar_dict = pickle.load(fo, encoding='bytes')
    return cifar_dict

In [4]:
dirs = ['batches.meta','data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5','test_batch']

In [5]:
all_data = [0,1,2,3,4,5,6]


In [6]:
for i,direc in zip(all_data,dirs):
    all_data[i] = unpickle(CIFAR_DIR+direc)


In [7]:
meta_data = all_data[0]
print(meta_data[b'label_names'])

[b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']


In [9]:
all_training_data = np.concatenate([all_data[1][b"data"] ,all_data[2][b"data"] ,all_data[3][b"data"] ,all_data[4][b"data"] ,all_data[5][b"data"] ])
all_training_labels = np.concatenate([all_data[1][b"labels"] ,all_data[2][b"labels"] ,all_data[3][b"labels"] ,all_data[4][b"labels"] ,all_data[5][b"labels"] ])

In [10]:
print(all_training_labels.shape)
print(len(all_training_labels))
print(all_training_labels.dtype)

(50000,)
50000
int64


In [11]:

def one_hot_encode(x):
    z = np.zeros((len(x), 10))
    z[list(np.indices((len(x),))) + [x]] = 1
    return z

all_training_labels = one_hot_encode(all_training_labels)



In [12]:
print(all_training_labels.shape)
print(len(all_training_labels))
print(all_training_labels.dtype)    


(50000, 10)
50000
float64


In [13]:
print(all_training_data.shape)

all_training_data = tf.reshape(all_training_data,shape=[50000, 3, 32, 32])

print(all_training_data.shape)

all_training_data = tf.transpose(all_training_data,perm=[0,2,3,1])

all_training_data = tf.cast(all_training_data,dtype=tf.float32)

(50000, 3072)
(50000, 3, 32, 32)


DEBUG::=> training data shapes
(50000, 32, 32, 3)
DEBUG::=> training labels shapes
(50000, 10)


In [15]:
data_dataset = tf.contrib.data.Dataset.from_tensor_slices(all_training_data)
labels_dataset = tf.contrib.data.Dataset.from_tensor_slices(all_training_labels)

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.


In [16]:
batched_data_dataset = data_dataset.batch(100)
batched_labels_dataset = labels_dataset.batch(100)

In [17]:
data_iterator = batched_data_dataset.make_initializable_iterator()
label_iterator = batched_labels_dataset.make_initializable_iterator()

In [18]:
print('DEBUG::=> data iterator output shapes')
print(data_iterator.output_shapes)
print('DEBUG::=> label iterator output shapes')
print(label_iterator.output_shapes)

DEBUG::=> data iterator output shapes
(?, 32, 32, 3)
DEBUG::=> label iterator output shapes
(?, 10)


# Model

In [19]:
#Hyper Parameter  
learning_rate = 0.01
num_epochs = 15

img_height = 32
img_width = 32
num_channels = 3 

#Network Parameters
num_input = 3072
num_classes = 1000
dropout_keep_prob = 0.75

In [20]:
def create_placeholders(height,width,num_channels,num_classes):
    X = tf.placeholder(dtype=tf.float32,shape=[None,32,32,3],name='X-data')
    Y = tf.placeholder(dtype=tf.float32,shape=[None,10],name='Y-labels')
    return X ,Y 

In [21]:

def initialize_parameters():

    # Create them 
    
    W1 = tf.Variable(tf.truncated_normal(shape=[3,3,3,32], stddev=0.05),name='W1')
    W2 = tf.Variable(tf.truncated_normal(shape=[3,3,32,64], stddev=0.05),name='W2')
    W3 = tf.Variable(tf.truncated_normal(shape=[3,3,64,32], stddev=0.05),name='W2')
    W4 = tf.Variable(tf.truncated_normal(shape=[3,3,32,16], stddev=0.05),name='W2')
    
    parameters = {"W1": W1,
                  "W2": W2,
                  "W3" :W3,
                  "W4": W4}
    
    return parameters

In [22]:
# Forward Pas
"""
    our forward pass through the network will consist of this 
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
"""
def forward_pass(X,weights):

    W1 = weights['W1']
    W2 = weights['W2']
    W3 = weights['W3']
    W4 = weights['W4']
    

    # CONV2D: stride of 1, padding 'SAME'
    Z1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME',name = 'Z1')
    # RELU
    A1 = tf.nn.relu(Z1,name = 'A1')
    # MAXPOOL: window 8x8, stride 8, padding 'SAME'
    P1 = tf.nn.max_pool(A1, ksize = [1, 8, 8, 1], strides = [1, 8, 8, 1], padding='SAME',name = 'P1')
    
    
    # CONV2D: filters W2, stride 1, padding 'SAME'
    Z2 = tf.nn.conv2d(P1, W2, strides=[1, 1, 1, 1], padding='SAME',name = 'Z2')
    # RELU
    A2 = tf.nn.relu(Z2,name = 'A2')
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    P2 = tf.nn.max_pool(A2, ksize = [1, 4, 4, 1], strides = [1, 4, 4, 1], padding='SAME',name = 'P2')
    
    
  
    Z3 = tf.nn.conv2d(P2, W3, strides=[1, 1, 1, 1], padding='SAME',name = 'Z3')
    # RELU
    A3 = tf.nn.relu(Z3,name = 'A3')
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    P3 = tf.nn.max_pool(A3, ksize = [1, 4, 4, 1], strides = [1, 4, 4, 1], padding='SAME',name = 'P3')
    
    
    Z4 = tf.nn.conv2d(P3, W4, strides=[1, 1, 1, 1], padding='SAME',name = 'Z4')
    # RELU
    A4 = tf.nn.relu(Z4,name = 'A4')
    # MAXPOOL: window 4x4, stride 4, padding 'SAME'
    P4 = tf.nn.max_pool(A4, ksize = [1, 4, 4, 1], strides = [1, 4, 4, 1], padding='SAME',name = 'P4')
    
    
    # FLATTEN
    P = tf.contrib.layers.flatten(P4)
    # FULLY-CONNECTED without non-linear activation function (not not call softmax).
    # 6 neurons in output layer. Hint: one of the arguments should be "activation_fn=None" 
    Z6 = tf.contrib.layers.fully_connected(P, 1000, activation_fn=None)
    Z7 = tf.contrib.layers.fully_connected(P, 512, activation_fn=None)
    Z8 = tf.contrib.layers.fully_connected(P, 256, activation_fn=None)
    Z9 = tf.contrib.layers.fully_connected(P, 128, activation_fn=None)
    Z10 = tf.contrib.layers.fully_connected(P, 64, activation_fn=None)
    Z11 = tf.contrib.layers.fully_connected(P, 32, activation_fn=None)    
    predictions = tf.contrib.layers.fully_connected(Z11,10,activation_fn=None)

    return predictions

In [23]:
# Now that we have Z3 and our true labels, let's compute the cost

def compute_cost(labels,Z6):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels,logits=Z6))
    return cost

In [24]:
# Finally let's create our model
losses = []
def model( num_epochs , learning_rate):
    #Create placeholder 

    X, Y = create_placeholders(img_height,img_width,num_channels,num_classes)

    parameters = initialize_parameters()
    predictions = forward_pass(X,parameters)
    print(parameters['W1'])
    summary_pred =  tf.summary.histogram(name = 'predictions',values=predictions)
    cost = compute_cost(Z6=predictions,labels=Y)
    summary_cost = tf.summary.histogram(name = 'cost',values=cost)    
    
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    init_data_iterator = data_iterator.initializer
    init_label_iterator = label_iterator.initializer
    train_writer = tf.summary.FileWriter('logs')
    summary_merger = tf.summary.merge_all()
    
    
    
    
    
    
    
    
    correct_pred = tf.equal(tf.argmax(predictions, 1), tf.argmax(all_training_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    
    
    
    
    
    
    init = tf.global_variables_initializer()
    with tf.Session() as sess :

        sess.run(init)

        sess.run(parameters)

        for epoch in range(1,num_epochs):

            sess.run(init_data_iterator)

            sess.run(init_label_iterator)

            for i in range(1,499):

                mini_batch_data =  sess.run(data_iterator.get_next())
      
                mini_batch_labels = sess.run(label_iterator.get_next())

                _ , loss_value,summary1,summary2,acc  = sess.run([train,cost,summary_pred,summary_cost,accuracy], feed_dict= {
                    X : mini_batch_data,
                    Y : mini_batch_labels
                })

                train_writer.add_summary(summary1)
                train_writer.add_summary(summary2)


                    
                    
                losses.append(loss_value)

                if i % 100 == 0 :
                    print('Epoch : {} minbatch {} loss_value {} accuract {}'.format(epoch,i , loss_value,acc))
                
            print('Epoch Done ')
            
        print('Training Done')
        print('Now Evaluating')



        
        train_writer.add_graph(sess.graph)

        

In [ ]:
model(learning_rate=0.01,num_epochs=2)

<tf.Variable 'W1:0' shape=(3, 3, 3, 32) dtype=float32_ref>
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
plt.plot(losses[300:400])

## Testing


In [ ]:
all_testing_data = all_data[6][b'data']
all_testing_labels = all_data[6][b'labels']

In [ ]:
print('DEBUG::=> Testing data shapes')
print('all_testing_data shape {}'.format(all_testing_data.shape))
print('all_testing_labels shape {}'.format(len(all_testing_labels)))

In [ ]:
all_testing_labels = one_hot_encode(all_testing_labels)

In [ ]:
all_testing_labels.shape


In [ ]:
print(all_testing_data.shape)

all_testing_data = tf.reshape(all_testing_data,shape=[10000, 3, 32, 32])

print(all_testing_data.shape)

all_testing_data = tf.transpose(all_testing_data,perm=[0,2,3,1])

all_testing_data = tf.cast(all_testing_data,dtype=tf.float32)